In [ ]:

import os, re, uuid
import pandas as pd
from typing import Dict, Any, List, Optional
from dataclasses import dataclass
from tqdm import tqdm
from dotenv import load_dotenv
from qdrant_client.models import Filter, FieldCondition, Range, MatchValue
from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, Range
from unidecode import unidecode

from typing import Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

from langchain_openai import OpenAIEmbeddings

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.109.1-py3-none-any.whl.metadata (29 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
Using cached openai-1.109.1-py3-none-any.whl (948 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [langchain-openai][openai]
Note: you may need to restart the kernel to use updated packages.


In [4]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [1]:
# Uğur burda df'i okut
import pandas as pd
df = pd.read_parquet("../data/arabam_ilanlar.parquet")

print("✅ Veri boyutu:", df.shape)
df.head()

✅ Veri boyutu: (65839, 20)


,id,baslik,konum,fiyat,aciklama,marka,seri,model,yil,kilometre,yakit_tipi,vites_tipi,renk,arac_durumu,kasa_tipi,cekis,motor_hacmi,motor_gucu,url,Araç_Yası
0,1,YAVUZLAR'DAN 2014 ALFA ROMEO GİULİETTA 1.6 JTD...,"Karşıyaka Mh. Kepez, Antalya",655000,\n,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,209000,Dizel,Düz,Beyaz,İkinci El,Hatchback/5,Önden Çekiş,1598,105,https://www.arabam.com/ilan/galeriden-satilik-...,11
1,2,2006 ALFA ROMEO 156 TS,"Soğanlı Mh. Osmangazi, Bursa",414500,ES ES OTOMOTİV DEN \n\n\n \n\n\nSATILIK \n\n\n...,Alfa Romeo,156,1.6 TS Distinctive,2006,171000,Benzin,Düz,Şampanya,İkinci El,Sedan,Önden Çekiş,1600,125,https://www.arabam.com/ilan/galeriden-satilik-...,19
2,3,Sahibinden Alfa Romeo Giulietta 1.4 TB MultiAi...,"Karşıyaka Mh. Karataş, Adana",735000,Ev almayı düşündüğüm için aracımı satışa çıkar...,Alfa Romeo,Giulietta,1.4 TB MultiAir Distinctive,2011,157100,Benzin,Düz,Siyah,İkinci El,Hatchback/5,Önden Çekiş,1368,170,https://www.arabam.com/ilan/sahibinden-satilik...,14
3,4,Sahibinden Alfa Romeo Giulietta 1.4 TB Progres...,"Esenkent Mh. Esenyurt, İstanbul",900000,"-2016 NİSAN ÇIKIŞLIDIR.-ORJİNAL 90 BİN KM, SIF...",Alfa Romeo,Giulietta,1.4 TB Progression Plus,2015,90800,Benzin,Düz,Kırmızı,İkinci El,Hatchback/5,Önden Çekiş,1368,120,https://www.arabam.com/ilan/sahibinden-satilik...,10
4,5,A L F İ S T,"Deliktaş Mh. Pamukkale, Denizli",800000,,Alfa Romeo,Giulietta,1.6 JTD Distinctive,2014,200000,Dizel,Düz,Beyaz,İkinci El,Hatchback/5,Önden Çekiş,1598,105,https://www.arabam.com/ilan/sahibinden-satilik...,11


In [6]:
def ascii_lower(s: str) -> str:
    return unidecode(str(s or "")).strip().lower()

def to_num(text) -> Optional[float]:
    if text is None: return None
    s = str(text).lower().replace("tl","").replace("₺","").replace("km","").strip()
    s = s.replace(".","").replace(" ","").replace(",",".")
    try: return float(s)
    except: return None

def year4(x) -> Optional[int]:
    m = re.search(r"\b(19|20)\d{2}\b", str(x))
    return int(m.group(0)) if m else None

def build_doc_text(row: Dict[str, Any]) -> str:
    """
    Embedding'e gidecek doğal dil "ilan profili".
    Kısa, tek satır ya da 2-3 cümle idealdir.
    """
    marka = row.get("marka","").strip()
    seri = row.get("seri","").strip()
    model = row.get("model","").strip()
    yil = row.get("yil") or row.get("yil_num")
    vites = row.get("vites_tipi","")
    yakit = row.get("yakit_tipi","")
    km = row.get("kilometre","")
    konum = row.get("konum","")
    kasa = row.get("kasa_tipi","")
    fiyat = row.get("fiyat","")
    aciklama = (row.get("aciklama") or "").strip()

    title = f"{marka} {seri} {model} {yil or ''}".strip()
    bullet = f"{yakit}, {vites}, {km} km, {kasa}, {konum}".replace("  "," ").strip(" ,")
    text = f"{title} – {bullet}. Fiyat: {fiyat}. {aciklama}"
    return " ".join(text.split())

In [7]:
def make_point_id(raw):
    """
    raw -> int (>=0) ya da UUID(str). Diğer her şey yeni UUID olur.
    """
    if raw is None:
        return str(uuid.uuid4())

    # önce integer dene
    try:
        # numpy int / float gelebilir
        if isinstance(raw, float):
            if math.isnan(raw):
                return str(uuid.uuid4())
            # 1.0 gibi ise int'e çevir
            if float(raw).is_integer() and int(raw) >= 0:
                return int(raw)
            else:
                return str(uuid.uuid4())
        iv = int(raw)
        if iv >= 0:
            return iv
    except Exception:
        pass

    # sonra UUID dene
    try:
        return str(uuid.UUID(str(raw)))
    except Exception:
        # en son yeni UUID üret
        return str(uuid.uuid4())

def df_to_points(df: pd.DataFrame, embedder: OpenAIEmbeddings, collection: str, client: QdrantClient, batch_size: int = 256):
    # Varsa önceden normalize edilmiş kolonları uygula:
    if "fiyat_num" not in df.columns:
        df["fiyat_num"] = df["fiyat"].map(to_num)
    if "km_num" not in df.columns:
        df["km_num"] = df["kilometre"].map(to_num)
    if "yil_num" not in df.columns:
        df["yil_num"] = df["yil"].map(year4)

    # Koleksiyon oluştur
    dim = embedder.dimension()  # encode("test") yerine güvenli yol

    # Koleksiyon yoksa oluştur:
    from qdrant_client.models import Distance, VectorParams
    if collection not in [c.name for c in client.get_collections().collections]:
        client.create_collection(
            collection_name=collection,
            vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
        )

    # Noktalar
    # TODO: Burdaki değişkenleri kontrol et
    rows = df.to_dict(orient="records")
    for i in range(0, len(rows), batch_size):
        chunk = rows[i:i+batch_size]
        texts = [build_doc_text(r) for r in chunk]
        vecs = embedder.embed_documents(texts)

        points = []
        for r, v, t in zip(chunk, vecs, texts):
            pid = make_point_id(r.get("id"))
            payload = {
                # ham alanlar
                "id": r.get("id"),
                "baslik": r.get("baslik"),
                "konum": r.get("konum"),
                "fiyat": r.get("fiyat"),
                "marka": r.get("marka"),
                "seri": r.get("seri"),
                "model": r.get("model"),
                "yil": r.get("yil_num") or r.get("yil"),
                "kilometre": r.get("km_num") or r.get("kilometre"),
                "yakit_tipi": r.get("yakit_tipi"),
                "vites_tipi": r.get("vites_tipi"),
                "kasa_tipi": r.get("kasa_tipi"),
                "arac_durumu": r.get("arac_durumu"),
                "cekis": r.get("cekis"),
                "motor_hacmi": r.get("motor_hacmi"),
                "motor_gucu": r.get("motor_gucu"),
                "tramer": r.get("tramer"),
                "url": r.get("url"),

                # sorguda filtre için sayısal/küçük harf anahtarlar
                "fiyat_num": r.get("fiyat_num"),
                "km_num": r.get("km_num"),
                "yil_num": r.get("yil_num"),
                "marka_key": ascii_lower(r.get("marka")),
                "seri_key": ascii_lower(r.get("seri")),
                "model_key": ascii_lower(r.get("model")),
                "konum_key": ascii_lower(r.get("konum")),

                # arama metni (dense + sparse için)
                "text": t,
            }
            points.append(PointStruct(id=pid, vector=v, payload=payload))

        client.upsert(collection_name=collection, points=points)

In [8]:
@dataclass
class QueryFilters:
    marka: Optional[str] = None
    seri: Optional[str] = None
    model: Optional[str] = None
    konum: Optional[str] = None
    fiyat_max: Optional[float] = None
    fiyat_min: Optional[float] = None
    yil_min: Optional[int] = None
    yil_max: Optional[int] = None

def build_qdrant_filter(f: QueryFilters) -> Filter | None:
    must: list = []

    def eq(field: str, val: str):
        val = (val or "").strip().lower()
        if val:
            must.append(FieldCondition(key=field, match=MatchValue(value=val)))

    def rng(field: str, gte=None, lte=None):
        cond = {}
        if gte is not None: cond["gte"] = float(gte)
        if lte is not None: cond["lte"] = float(lte)
        if cond:
            must.append(FieldCondition(key=field, range=Range(**cond)))

    eq("marka_key", f.marka)
    eq("seri_key",  f.seri)
    eq("model_key", f.model)
    eq("konum_key", f.konum)

    rng("fiyat_num", gte=f.fiyat_min, lte=f.fiyat_max)
    rng("yil_num",   gte=f.yil_min,   lte=f.yil_max)

    return Filter(must=must) if must else None

def _is_valid_text(s: str, min_len: int = 3) -> bool:
    return isinstance(s, str) and len(s.strip()) >= min_len

class HybridSearcher:
    def __init__(self, client, collection: str, embedder):
        self.client = client
        self.collection = collection
        self.embedder = embedder
        self._tfidf: TfidfVectorizer | None = None
        self._sparse = None
        self._ids: np.ndarray | None = None
        self._texts: List[str] | None = None

    def _ensure_sparse_index(self, max_points: int = 20000):
        if self._tfidf is not None:
            return

        texts: List[str] = []
        ids:   List[str] = []

        # Qdrant scroll ile sayfalama
        next_offset = None
        seen = set()
        while True:
            recs, next_offset = self.client.scroll(
                collection_name=self.collection,
                with_payload=True,
                limit=1024,
                offset=next_offset
            )

            if not recs:
                break

            for p in recs:
                pid = str(p.id)
                if pid in seen:
                    continue
                pl = p.payload or {}
                t = pl.get("text", "")
                if _is_valid_text(t):
                    texts.append(t)
                    ids.append(pid)
                    seen.add(pid)

            if next_offset is None or len(texts) >= max_points:
                break

        # boş kalırsa TF-IDF fit patlamasın
        if not texts:
            # Minimum bir dummy belge ekleyerek fit'i güvene al
            texts = ["placeholder"]
            ids   = ["__placeholder__"]

        self._tfidf = TfidfVectorizer(max_features=50000, ngram_range=(1, 2))
        self._sparse = self._tfidf.fit_transform(texts)
        self._ids = np.array(ids)
        self._texts = texts

    def dense_search(self, query: str, f: Optional[QueryFilters], top_k: int = 50):
        qv = self.embedder.embed_query(query)
        qf = build_qdrant_filter(f) if f else None
        res = self.client.search(
            collection_name=self.collection,
            query_vector=qv,
            query_filter=qf,
            limit=top_k,
            with_payload=True,
            with_vectors=False
        )
        return [(str(r.id), float(r.score), r.payload) for r in res]

    def sparse_search(self, query: str, f: Optional[QueryFilters], top_k: int = 200):
        self._ensure_sparse_index()
        q = self._tfidf.transform([query])                 # csr_matrix (1 x V)
        # TF-IDF default: l2 normalize; dot product ≈ cosine
        sim = (q @ self._sparse.T).toarray().ravel()       # ndarray (N,)

        # En hızlı top-k (tam sıralama yerine)
        k = min(top_k, sim.size)
        if k == 0:
            return []
        idx = np.argpartition(-sim, k-1)[:k]               # ilk k skorun indeksleri (sırasız)
        order = idx[np.argsort(-sim[idx])]                 # skora göre sırala

        results = []
        for i in order:
            pid = self._ids[i]
            sc  = float(sim[i])

            # Point + payload çek
            pts = self.client.retrieve(self.collection, ids=[pid], with_payload=True)
            if not pts:
                continue
            pl = pts[0].payload or {}

            # Payload filtrelerini uygula (Qdrant filter sparse'ta kullanılmadı)
            if f:
                if f.marka and (pl.get("marka_key") or "") != (f.marka or "").strip().lower():  continue
                if f.seri  and (pl.get("seri_key")  or "") != (f.seri  or "").strip().lower():  continue
                if f.model and (pl.get("model_key") or "") != (f.model or "").strip().lower():  continue
                if f.konum and (pl.get("konum_key") or "") != (f.konum or "").strip().lower():  continue
                if f.fiyat_min is not None and (pl.get("fiyat_num") is None or pl["fiyat_num"] < f.fiyat_min): continue
                if f.fiyat_max is not None and (pl.get("fiyat_num") is None or pl["fiyat_num"] > f.fiyat_max): continue
                if f.yil_min  is not None and (pl.get("yil_num")   is None or pl["yil_num"]   < f.yil_min):  continue
                if f.yil_max  is not None and (pl.get("yil_num")   is None or pl["yil_num"]   > f.yil_max):  continue

            results.append((pid, sc, pl))
        return results


    def rrf_merge(self, dense, sparse, k: float = 60.0, top_k: int = 50):
        # dense/sparse: [(id, score, payload)]
        # RRF için önce id->rank
        def ranks(lst):
            return {pid: rank for rank, (pid, _, _) in enumerate(sorted(lst, key=lambda x: -x[1]), start=1)}
        rd = ranks(dense)
        rs = ranks(sparse)
        ids = set([pid for pid,_,_ in dense] + [pid for pid,_,_ in sparse])
        merged = []
        for pid in ids:
            r1 = rd.get(pid, 10**6)
            r2 = rs.get(pid, 10**6)
            rrf = 1.0/(k+r1) + 1.0/(k+r2)
            payload = None
            # payload çek
            if pid in rd:
                payload = [pl for (p,_,pl) in dense if p==pid][0]
            elif pid in rs:
                payload = [pl for (p,_,pl) in sparse if p==pid][0]
            merged.append((pid, rrf, payload))
        merged.sort(key=lambda x: -x[1])
        return merged[:top_k]

    def search(self, query_text: str, f: Optional[QueryFilters] = None, top_k: int = 30):
        dense = self.dense_search(query_text, f, top_k=top_k)
        sparse = self.sparse_search(query_text, f, top_k=top_k*4)
        merged = self.rrf_merge(dense, sparse, top_k=top_k)
        return merged

In [9]:
class ST_Embedder:
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                 force_device: str = "cpu"):  # "cpu" veya "cuda"
        self.device = force_device
        self.model = SentenceTransformer(model_name, device=self.device)  # CPU'ya zorladık

    def embed_documents(self, texts):
        try:
            return self.model.encode(
                texts,
                convert_to_numpy=True,
                batch_size=64,
                normalize_embeddings=True,   # cosine için iyi pratik
                show_progress_bar=False
            ).tolist()
        except Exception:
            # GPU'da hata olursa otomatik CPU'ya düş
            if self.device != "cpu":
                self.device = "cpu"
                self.model = SentenceTransformer(self.model._model_card, device="cpu")
                arr = self.model.encode(
                    texts, convert_to_numpy=True, batch_size=64, normalize_embeddings=True, show_progress_bar=False
                ).tolist()
                return arr
            raise

    def embed_query(self, text: str):
        vec = self.embed_documents([text])[0]
        return vec

    def dimension(self) -> int:
        # SBERT modelleri bu özelliği sağlar; yoksa örnekle öğren
        if hasattr(self.model, "get_sentence_embedding_dimension"):
            return self.model.get_sentence_embedding_dimension()
        return len(self.embed_query("test"))

In [ ]:
# car_vector_search.py
# Gereksinimler:
# pip install pandas qdrant-client sentence-transformers scikit-learn unidecode pydantic tqdm

from __future__ import annotations
import os
import re
import math
import uuid
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple

import pandas as pd
import numpy as np
from tqdm import tqdm
from unidecode import unidecode
from pydantic import BaseModel, Field

# Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    VectorParams,
    PointStruct,
    Filter,
    FieldCondition,
    Range,
    MatchValue,
)

# -----------------------------
# 0) Embedder (Sentence-Transformers - CPU varsayılan)
# -----------------------------
from sentence_transformers import SentenceTransformer

class ST_Embedder:
    """
    Sentence-Transformers tabanlı embedder.
    - Varsayılan cihaz: CPU
    - normalize_embeddings=True (cosine için iyi pratik)
    """
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                 force_device: str = "cpu"):
        self.device = force_device
        self.model_name = model_name
        self.model = SentenceTransformer(model_name, device=self.device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        try:
            return self.model.encode(
                texts,
                convert_to_numpy=True,
                batch_size=64,
                normalize_embeddings=True,
                show_progress_bar=False,
            ).tolist()
        except Exception:
            # GPU'da sorun olursa CPU'ya düş
            if self.device != "cpu":
                self.device = "cpu"
                self.model = SentenceTransformer(self.model_name, device="cpu")
                return self.model.encode(
                    texts,
                    convert_to_numpy=True,
                    batch_size=64,
                    normalize_embeddings=True,
                    show_progress_bar=False,
                ).tolist()
            raise

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

    def dimension(self) -> int:
        if hasattr(self.model, "get_sentence_embedding_dimension"):
            return self.model.get_sentence_embedding_dimension()
        # çok nadir durumda:
        return len(self.embed_query("test"))


# -----------------------------
# 1) Yardımcılar / Normalizasyon
# -----------------------------
TURKISH_MAP = {
    "otomatik": ["otomatik", "auto", "dct", "edc", "e-cvt", "cvt", "tiptronic", "multitronic", "dsg"],
    "manuel": ["manuel", "manual"],
    "benzin": ["benzin", "gasoline", "benzinli"],
    "dizel": ["dizel", "diesel"],
    "lpg": ["lpg", "autogas"],
    "hybrid": ["hibrid", "hybrid"],
    "elektrik": ["elektrik", "electric", "bev", "ev"],
}

def ascii_lower(s: Any) -> str:
    return unidecode(str(s or "")).strip().lower()

def to_num(text: Any) -> Optional[float]:
    if text is None:
        return None
    s = str(text).strip().lower()
    if s in ("nan", "", "none", "yok", "—", "-"):
        return None
    s = s.replace("tl", "").replace("₺", "").replace("km", "")
    s = s.replace("milyon", "000000").replace("mn", "000000").replace("m", "000000")
    s = re.sub(
        r"(\d+)[\.,]?(\d*)\s*bin",
        lambda m: str(float(m.group(1) + "." + (m.group(2) or "0")) * 1000),
        s,
    )
    s = s.replace(".", "").replace(" ", "")
    s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return None

def year4(x: Any) -> Optional[int]:
    m = re.search(r"\b(19|20)\d{2}\b", str(x or ""))
    return int(m.group(0)) if m else None

def none_if_nan(x: Any):
    try:
        return None if (x is None or (isinstance(x, float) and math.isnan(x))) else x
    except:
        return x

def make_point_id(raw: Any):
    """
    Qdrant ID: unsigned int veya UUID string olmalı.
    """
    if raw is None:
        return str(uuid.uuid4())
    # int?
    try:
        if isinstance(raw, float):
            if math.isnan(raw):
                return str(uuid.uuid4())
            if float(raw).is_integer() and int(raw) >= 0:
                return int(raw)
            return str(uuid.uuid4())
        iv = int(raw)
        if iv >= 0:
            return iv
    except Exception:
        pass
    # UUID?
    try:
        return str(uuid.UUID(str(raw)))
    except Exception:
        return str(uuid.uuid4())

def match_from_map(value: str, mapping: Dict[str, List[str]]) -> str:
    v = ascii_lower(value)
    for canon, variants in mapping.items():
        for t in variants:
            if t in v:
                return canon
    return value


# -----------------------------
# 2) DataFrame Normalize
# -----------------------------
def normalize_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    expected = [
        "id","baslik","konum","fiyat","aciklama","marka","seri","model","yil","kilometre",
        "yakit_tipi","vites_tipi","renk","arac_durumu","kasa_tipi","cekis","motor_hacmi",
        "motor_gucu","tramer","url"
    ]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(f"Eksik kolon(lar): {missing}")

    # metin
    for col in [
        "baslik","konum","aciklama","marka","seri","model","yakit_tipi","vites_tipi",
        "renk","arac_durumu","kasa_tipi","cekis","url"
    ]:
        df[col] = df[col].map(lambda x: re.sub(r"\s+", " ", str(x or "").strip()))

    # sayısal
    df["fiyat_num"] = df["fiyat"].map(to_num)
    df["km_num"] = df["kilometre"].map(to_num)
    df["tramer_num"] = df["tramer"].map(to_num)
    df["yil_num"] = df["yil"].map(year4)

    # vites / yakıt std
    df["vites_std"] = df["vites_tipi"].apply(
        lambda v: match_from_map(v, {"otomatik": TURKISH_MAP["otomatik"], "manuel": TURKISH_MAP["manuel"]})
    )
    def yakit_std(v):
        if match_from_map(v, {"benzin": TURKISH_MAP["benzin"]}) == "benzin": return "benzin"
        if match_from_map(v, {"dizel": TURKISH_MAP["dizel"]}) == "dizel": return "dizel"
        if match_from_map(v, {"lpg": TURKISH_MAP["lpg"]}) == "lpg": return "lpg"
        if match_from_map(v, {"hybrid": TURKISH_MAP["hybrid"]}) == "hybrid": return "hybrid"
        if match_from_map(v, {"elektrik": TURKISH_MAP["elektrik"]}) == "elektrik": return "elektrik"
        return ascii_lower(v)
    df["yakit_std"] = df["yakit_tipi"].map(yakit_std)

    # arama anahtarları
    for col in ["marka","seri","model","konum","kasa_tipi","cekis","renk","arac_durumu"]:
        df[col + "_key"] = df[col].map(ascii_lower)

    return df


# -----------------------------
# 3) Doc metni + payload
# -----------------------------
def build_doc_text(row: Dict[str, Any]) -> str:
    marka = str(row.get("marka", "")).strip()
    seri = str(row.get("seri", "")).strip()
    model = str(row.get("model", "")).strip()
    yil = row.get("yil_num") or row.get("yil") or ""
    vites = row.get("vites_tipi", "")
    yakit = row.get("yakit_tipi", "")
    km = row.get("kilometre", "")
    konum = row.get("konum", "")
    kasa = row.get("kasa_tipi", "")
    fiyat = row.get("fiyat", "")
    aciklama = (row.get("aciklama") or "").strip()

    title = f"{marka} {seri} {model} {yil}".strip()
    bullet = f"{yakit}, {vites}, {km} km, {kasa}, {konum}".replace("  "," ").strip(" ,")
    text = f"{title} – {bullet}. Fiyat: {fiyat}. {aciklama}"
    return " ".join(text.split())

def build_payload(r: Dict[str, Any], text: str) -> Dict[str, Any]:
    return {
        # ham alanlar
        "id": r.get("id"),
        "baslik": r.get("baslik"),
        "konum": r.get("konum"),
        "fiyat": r.get("fiyat"),
        "marka": r.get("marka"),
        "seri": r.get("seri"),
        "model": r.get("model"),
        "yil": r.get("yil_num") or r.get("yil"),
        "kilometre": r.get("km_num") or r.get("kilometre"),
        "yakit_tipi": r.get("yakit_tipi"),
        "vites_tipi": r.get("vites_tipi"),
        "renk": r.get("renk"),
        "arac_durumu": r.get("arac_durumu"),
        "kasa_tipi": r.get("kasa_tipi"),
        "cekis": r.get("cekis"),
        "motor_hacmi": r.get("motor_hacmi"),
        "motor_gucu": r.get("motor_gucu"),
        "tramer": r.get("tramer"),
        "url": r.get("url"),

        # sayısal & key alanlar
        "fiyat_num": none_if_nan(r.get("fiyat_num")),
        "km_num": none_if_nan(r.get("km_num")),
        "yil_num": none_if_nan(r.get("yil_num")),
        "marka_key": ascii_lower(r.get("marka")),
        "seri_key": ascii_lower(r.get("seri")),
        "model_key": ascii_lower(r.get("model")),
        "konum_key": ascii_lower(r.get("konum")),
        "kasa_tipi_key": ascii_lower(r.get("kasa_tipi")),
        "cekis_key": ascii_lower(r.get("cekis")),
        "renk_key": ascii_lower(r.get("renk")),
        "arac_durumu_key": ascii_lower(r.get("arac_durumu")),

        # arama metni
        "text": text,
    }



# -----------------------------
# 4) Qdrant collection garanti
# -----------------------------
def ensure_collection(client: QdrantClient, collection: str, dim: int, distance: Distance = Distance.COSINE):
    existing = [c.name for c in client.get_collections().collections]
    if collection in existing:
        # Boyut uyuşmasını kontrol et (Qdrant sürümüne göre alanlar değişebilir)
        info = client.get_collection(collection)
        # Bazı sürümlerde: info.config.params.vectors.size
        current_dim = None
        try:
            current_dim = info.config.params.vectors.size  # type: ignore
        except Exception:
            # Yedek yol: vektör sayısı boyut değil, o yüzden kullanma
            pass
        if current_dim is not None and current_dim != dim:
            raise ValueError(f"Koleksiyon '{collection}' farklı boyutta: {current_dim} ≠ {dim}")
        return

    client.create_collection(
        collection_name=collection,
        vectors_config=VectorParams(size=dim, distance=distance),
    )


# -----------------------------
# 5) DF → Qdrant Upsert
# -----------------------------
def df_to_points(df: pd.DataFrame, embedder: ST_Embedder, collection: str,
                 client: QdrantClient, batch_size: int = 256):
    if "fiyat_num" not in df.columns:
        df["fiyat_num"] = df["fiyat"].map(to_num)
    if "km_num" not in df.columns:
        df["km_num"] = df["kilometre"].map(to_num)
    if "yil_num" not in df.columns:
        df["yil_num"] = df["yil"].map(year4)

    dim = embedder.dimension()
    ensure_collection(client, collection, dim)

    rows = df.to_dict(orient="records")
    for i in tqdm(range(0, len(rows), batch_size), desc="upserting"):
        chunk = rows[i:i + batch_size]
        texts = [build_doc_text(r) for r in chunk]
        vecs = embedder.embed_documents(texts)

        points = []
        for r, v, t in zip(chunk, vecs, texts):
            pid = make_point_id(r.get("id"))
            payload = build_payload(r, t)
            points.append(PointStruct(id=pid, vector=v, payload=payload))

        client.upsert(collection_name=collection, points=points)


# -----------------------------
# 6) Filtre modeli (sorgu → payload filter)
# -----------------------------
class QueryFilters(BaseModel):
    marka: Optional[str] = None
    seri: Optional[str] = None
    model: Optional[str] = None
    konum: Optional[str] = None
    fiyat_min: Optional[float] = None
    fiyat_max: Optional[float] = None
    yil_min: Optional[int] = None
    yil_max: Optional[int] = None

def build_qdrant_filter(f: QueryFilters) -> Optional[Filter]:
    must: List[FieldCondition] = []

    def eq(field: str, val: Optional[str]):
        val = (val or "").strip().lower()
        if val:
            must.append(FieldCondition(key=field, match=MatchValue(value=val)))

    def rng(field: str, gte=None, lte=None):
        cond = {}
        if gte is not None: cond["gte"] = float(gte)
        if lte is not None: cond["lte"] = float(lte)
        if cond:
            must.append(FieldCondition(key=field, range=Range(**cond)))

    eq("marka_key", f.marka)
    eq("seri_key",  f.seri)
    eq("model_key", f.model)
    eq("konum_key", f.konum)
    rng("fiyat_num", gte=f.fiyat_min, lte=f.fiyat_max)
    rng("yil_num",   gte=f.yil_min,   lte=f.yil_max)

    return Filter(must=must) if must else None


# -----------------------------
# 7) HybridSearcher (Dense + Sparse TF-IDF + RRF)
# -----------------------------
from sklearn.feature_extraction.text import TfidfVectorizer

def _is_valid_text(s: str, min_len: int = 3) -> bool:
    return isinstance(s, str) and len(s.strip()) >= min_len

class HybridSearcher:
    def __init__(self, client: QdrantClient, collection: str, embedder: ST_Embedder):
        self.client = client
        self.collection = collection
        self.embedder = embedder
        self._tfidf: Optional[TfidfVectorizer] = None
        self._sparse = None
        self._ids: Optional[np.ndarray] = None   # dtype=object
        self._texts: Optional[List[str]] = None
        self._payloads: Dict[Any, Dict] = {}     # id -> payload (cache)

    def _ensure_sparse_index(self, max_points: int = 20000):
        if self._tfidf is not None:
            return

        texts: List[str] = []
        ids:   List[Any] = []
        seen = set()
        next_offset = None

        while True:
            recs, next_offset = self.client.scroll(
                collection_name=self.collection,
                with_payload=True,
                limit=1024,
                offset=next_offset,
            )
            if not recs:
                break

            for p in recs:
                pid = p.id  # tipini KORU (int ya da str-UUID)
                if pid in seen:
                    continue
                pl = p.payload or {}
                t = pl.get("text", "")
                if _is_valid_text(t):
                    texts.append(t)
                    ids.append(pid)
                    self._payloads[pid] = pl
                    seen.add(pid)

            if next_offset is None or len(texts) >= max_points:
                break

        if not texts:
            texts = ["placeholder"]
            ids = ["__placeholder__"]
            self._payloads["__placeholder__"] = {"text": "placeholder"}

        self._tfidf = TfidfVectorizer(max_features=50000, ngram_range=(1, 2))
        self._sparse = self._tfidf.fit_transform(texts)
        self._ids = np.array(ids, dtype=object)
        self._texts = texts

    def dense_search(self, query: str, f: Optional[QueryFilters], top_k: int = 50):
        qv = self.embedder.embed_query(query)
        qf = build_qdrant_filter(f) if f else None
        res = self.client.search(
            collection_name=self.collection,
            query_vector=qv,
            query_filter=qf,
            limit=top_k,
            with_payload=True,
            with_vectors=False,
        )
        return [(r.id, float(r.score), r.payload or {}) for r in res]

    def sparse_search(self, query: str, f: Optional[QueryFilters], top_k: int = 200):
        self._ensure_sparse_index()
        q = self._tfidf.transform([query])
        sim = (q @ self._sparse.T).toarray().ravel()

        k = min(top_k, sim.size)
        if k == 0:
            return []
        idx = np.argpartition(-sim, k - 1)[:k]
        order = idx[np.argsort(-sim[idx])]

        results = []
        for i in order:
            pid = self._ids[i]
            sc = float(sim[i])
            pl = self._payloads.get(pid, {})

            # Payload filtreleri
            if f:
                if f.marka and (pl.get("marka_key") or "") != (f.marka or "").strip().lower():  continue
                if f.seri  and (pl.get("seri_key")  or "") != (f.seri  or "").strip().lower():  continue
                if f.model and (pl.get("model_key") or "") != (f.model or "").strip().lower():  continue
                if f.konum and (pl.get("konum_key") or "") != (f.konum or "").strip().lower():  continue
                if f.fiyat_min is not None and (pl.get("fiyat_num") is None or pl["fiyat_num"] < f.fiyat_min): continue
                if f.fiyat_max is not None and (pl.get("fiyat_num") is None or pl["fiyat_num"] > f.fiyat_max): continue
                if f.yil_min  is not None and (pl.get("yil_num")   is None or pl["yil_num"]   < f.yil_min):  continue
                if f.yil_max  is not None and (pl.get("yil_num")   is None or pl["yil_num"]   > f.yil_max):  continue

            results.append((pid, sc, pl))
        return results

    @staticmethod
    def rrf_merge(dense: List[Tuple[Any, float, Dict]], sparse: List[Tuple[Any, float, Dict]],
                  k: float = 60.0, top_k: int = 50):
        def ranks(lst):
            return {pid: rank for rank, (pid, _, _) in enumerate(sorted(lst, key=lambda x: -x[1]), start=1)}
        rd = ranks(dense)
        rs = ranks(sparse)
        ids = set([pid for pid, _, _ in dense] + [pid for pid, _, _ in sparse])
        merged = []
        for pid in ids:
            r1 = rd.get(pid, 10**6)
            r2 = rs.get(pid, 10**6)
            rrf = 1.0 / (k + r1) + 1.0 / (k + r2)
            payload = None
            if pid in rd:
                payload = [pl for (p, _, pl) in dense if p == pid][0]
            elif pid in rs:
                payload = [pl for (p, _, pl) in sparse if p == pid][0]
            merged.append((pid, rrf, payload or {}))
        merged.sort(key=lambda x: -x[1])
        return merged[:top_k]

    def search(self, query_text: str, f: Optional[QueryFilters] = None, top_k: int = 30):
        dense = self.dense_search(query_text, f, top_k=top_k)
        sparse = self.sparse_search(query_text, f, top_k=top_k * 4)
        return self.rrf_merge(dense, sparse, top_k=top_k)


# -----------------------------
# 9) Örnek kullanım (çalıştırmayın)
# -----------------------------
"""
# 0) Qdrant bağlantısı
client = QdrantClient(url="http://localhost:6333", prefer_grpc=False)

# 1) DataFrame yükle ve normalize et
df = pd.read_parquet("ilanlar.parquet")   # veya csv
df = normalize_df(df)

# 2) Embedder (CPU)
embedder = ST_Embedder("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", force_device="cpu")

# 3) Upsert
collection = "car_listings_st"   # 384-dim modeller için ayrı koleksiyon önerilir
df_to_points(df, embedder, collection, client, batch_size=256)

# 4) Arama
searcher = HybridSearcher(client, collection, embedder)
user_query = "İstanbul’da 1.3 milyon TL’ye kadar, 2018 üzeri otomatik benzinli Astra"
filters = heuristic_parse(user_query)  # veya LLM tabanlı parser
results = searcher.search(user_query, filters, top_k=20)

# results: [(id, score, payload), ...]
# payload['marka'], payload['model'], payload['yil_num'], payload['fiyat_num'], payload['url'] ...
"""


'\n# 0) Qdrant bağlantısı\nclient = QdrantClient(url="http://localhost:6333", prefer_grpc=False)\n\n# 1) DataFrame yükle ve normalize et\ndf = pd.read_parquet("ilanlar.parquet")   # veya csv\ndf = normalize_df(df)\n\n# 2) Embedder (CPU)\nembedder = ST_Embedder("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", force_device="cpu")\n\n# 3) Upsert\ncollection = "car_listings_st"   # 384-dim modeller için ayrı koleksiyon önerilir\ndf_to_points(df, embedder, collection, client, batch_size=256)\n\n# 4) Arama\nsearcher = HybridSearcher(client, collection, embedder)\nuser_query = "İstanbul’da 1.3 milyon TL’ye kadar, 2018 üzeri otomatik benzinli Astra"\nfilters = heuristic_parse(user_query)  # veya LLM tabanlı parser\nresults = searcher.search(user_query, filters, top_k=20)\n\n# results: [(id, score, payload), ...]\n# payload[\'marka\'], payload[\'model\'], payload[\'yil_num\'], payload[\'fiyat_num\'], payload[\'url\'] ...\n'

In [11]:
client = QdrantClient(url="http://localhost:6333", prefer_grpc=False)

In [13]:
embedder = ST_Embedder("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", force_device="cpu")

In [14]:
collection = "car_listings_st"   # 384-dim modeller için ayrı koleksiyon önerilir
df_to_points(df, embedder, collection, client, batch_size=256)

upserting: 100%|██████████| 258/258 [12:47<00:00,  2.98s/it]


In [ ]:
# ============================
# LLM tabanlı filtre çıkarma - DÜZELTİLMİŞ VERSİYON
# ============================
from typing import Optional, List, Dict, Any, Union
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from qdrant_client.models import Filter as QFilter, FieldCondition, MatchValue, Range

class RangeSpec(BaseModel):
    gte: Optional[float] = None
    lte: Optional[float] = None

class FilterSpec(BaseModel):
    must: Dict[str, Union[str, List[str]]] = Field(default_factory=dict)
    must_not: Dict[str, Union[str, List[str]]] = Field(default_factory=dict)
    should: Dict[str, List[str]] = Field(default_factory=dict)
    ranges: Dict[str, RangeSpec] = Field(default_factory=dict)

# Alan adları eşlemesi (LLM yanlış anahtar verirse düzeltmek için)
FIELD_MAP = {
    # metin-eşitlik anahtarları
    "marka": "marka_key",
    "seri": "seri_key",
    "model": "model_key",
    "konum": "konum_key",
    "kasa": "kasa_tipi_key",
    "kasa_tipi": "kasa_tipi_key",
    "cekis": "cekis_key",
    "renk": "renk_key",
    "durum": "arac_durumu_key",
    "arac_durumu": "arac_durumu_key",
    "vites": "vites_std",
    "yakit": "yakit_std",

    # sayısal/aritmetik anahtarlar
    "fiyat": "fiyat_num",
    "fiyat_num": "fiyat_num",
    "km": "km_num",
    "kilometre": "km_num",
    "km_num": "km_num",
    "yil": "yil_num",
    "yil_num": "yil_num",
    "tramer": "tramer_num",
    "tramer_num": "tramer_num",
    "arac_yasi": "Araç_Yası",
    "araç_yası": "Araç_Yası",
}

# değer normalizasyonu
VALUE_NORMALIZATION = {
    "vites_std": {
        "otomatik": ["otomatik", "auto", "dct", "edc", "e-cvt", "cvt", "tiptronic", "multitronic", "dsg"],
        "manuel":   ["manuel", "manual"],
    },
    "yakit_std": {
        "benzin":   ["benzin", "benzinli", "gasoline"],
        "dizel":    ["dizel", "diesel"],
        "lpg":      ["lpg", "autogas"],
        "hybrid":   ["hibrid", "hybrid"],
        "elektrik": ["elektrik", "electric", "bev", "ev"],
    },
}

def _canon_value(field: str, val: str) -> str:
    from unidecode import unidecode  # Bu import'u eklemeyi unutmayın
    v = unidecode(str(val or "")).strip().lower()
    mapping = VALUE_NORMALIZATION.get(field)
    if not mapping:
        return v
    for canon, variants in mapping.items():
        for t in variants:
            if t in v:
                return canon
    return v

# Prompt parçaları - SÜSLü PARANTEZLER ESCAPELENDİ
SCHEMA = (
    '{{\n'
    '  "must": {{ "<field>": "deger" | ["deger1", "deger2"] }},\n'
    '  "must_not": {{ "<field>": "deger" | ["..."] }},\n'
    '  "should": {{ "<field>": ["degerA","degerB"] }},\n'
    '  "ranges": {{ "<numeric_field>": {{ "gte": <float?>, "lte": <float?> }} }}\n'
    '}}\n'
)

FEWSHOT = (
    '[\n'
    '  {{\n'
    '    "must": {{\n'
    '      "konum_key": "istanbul",\n'
    '      "marka_key": "opel",\n'
    '      "seri_key": "astra",\n'
    '      "vites_std": "otomatik",\n'
    '      "yakit_std": "benzin"\n'
    '    }},\n'
    '    "must_not": {{}},\n'
    '    "should": {{}},\n'
    '    "ranges": {{\n'
    '      "fiyat_num": {{ "gte": null, "lte": 1300000 }},\n'
    '      "yil_num":   {{ "gte": 2019, "lte": null }}\n'
    '    }}\n'
    '  }},\n'
    '  {{\n'
    '    "must": {{ "yakit_std": "dizel" }},\n'
    '    "must_not": {{}},\n'
    '    "should": {{ "konum_key": ["ankara","izmir"] }},\n'
    '    "ranges": {{\n'
    '      "yil_num": {{ "gte": 2016, "lte": 2020 }},\n'
    '      "km_num":  {{ "gte": null, "lte": 120000 }}\n'
    '    }}\n'
    '  }}\n'
    ']'
)

FILTER_SYSTEM = (
    "Sen bir araç arama sorgusu ayrıştırıcısısın.\n"
    "Görev: Kullanıcının doğal dildeki arama niyetinden yapısal bir filtre JSON'u çıkar.\n\n"
    "Kurallar:\n"
    "- SADECE TEK BİR JSON OBJESI ver. Liste değil, tek obje. Açıklama yazma.\n"
    "- Şema aşağıdaki gibidir:\n"
    "{SCHEMA}\n"
    "- Alan adları payload alanlarına hizalı olmalı (örnekler):\n"
    "  marka_key, seri_key, model_key, konum_key, kasa_tipi_key, cekis_key, renk_key, arac_durumu_key, vites_std, yakit_std,\n"
    "  fiyat_num, km_num, yil_num, tramer_num, Araç_Yası\n"
    "- Eşanlamlıları standardize et: vites_std ∈ {{\"otomatik\",\"manuel\"}}; yakit_std ∈ {{\"benzin\",\"dizel\",\"lpg\",\"hybrid\",\"elektrik\"}}.\n"
    "- \"üstü\", \"altı\", \"en az\", \"en çok\", \"arası\", \"veya\" gibi kalıpları uygun range/list biçimine çevir.\n"
    "- Marka/seri/model/konum gibi isimleri *_key alanlarına küçük harf/ASCII normalize et.\n"
    "- Belirsizse alanı ekleme. JSON dışına çıkma.\n"
    "- ÖNEMLİ: Sadece son sorgu için (3. sorgu) JSON üret, örnekleri dahil etme!"
)

FILTER_HUMAN = (
    "İlk iki sorgu örnek, sadece üçüncü sorgu için JSON üret:\n\n"
    "ÖRNEK 1: \"İstanbul'da 1.3 milyon TL'ye kadar, 2018 üzeri otomatik benzinli Opel Astra\"\n"
    "ÖRNEK 2: \"Ankara veya İzmir, 2016-2020 arası, km en çok 120 bin, dizel\"\n\n"
    "ŞİMDİ BU SORGU İÇİN JSON ÜRET: \"{user_query}\"\n\n"
    "Örnek çıktı formatı (sadece referans, kopyalama):\n"
    "{FEWSHOT}\n\n"
    "SADECE aşağıdaki sorgu için tek bir JSON objesi üret:\n"
    "SORGU: \"{user_query}\"\n\n"
    "{format_instructions}"
)

filter_prompt = ChatPromptTemplate.from_messages([
    ("system", FILTER_SYSTEM),
    ("human", FILTER_HUMAN),
]).partial(SCHEMA=SCHEMA, FEWSHOT=FEWSHOT)

filter_parser = PydanticOutputParser(pydantic_object=FilterSpec)

def llm_parse_filters_all_fields(
    api_key: str,
    user_query: str,
    model_name: str = "gpt-4o-mini",
    temperature: float = 0.0
) -> FilterSpec:
    llm = ChatOpenAI(api_key=api_key, model=model_name, temperature=temperature)
    chain = filter_prompt | llm
    
    # İlk önce raw response alalım
    response = chain.invoke({
        "user_query": user_query,
        "format_instructions": filter_parser.get_format_instructions()
    })
    
    # Response'u temizleyelim
    import json
    import re
    
    content = response.content if hasattr(response, 'content') else str(response)
    
    # Eğer response bir liste içeriyorsa, son elemanı alalım
    try:
        parsed = json.loads(content)
        if isinstance(parsed, list):
            # Liste ise son elemanı (kullanıcının sorgusuna ait olanı) alalım
            content = json.dumps(parsed[-1])
    except:
        pass
    
    # JSON dışındaki metinleri temizle
    json_match = re.search(r'\{.*\}', content, re.DOTALL)
    if json_match:
        content = json_match.group()
    
    # Şimdi parse edelim
    try:
        json_obj = json.loads(content)
        spec = FilterSpec.model_validate(json_obj)
    except Exception as e:
        print(f"Parse hatası: {e}")
        print(f"Raw content: {content}")
        # Fallback: boş FilterSpec döndür
        spec = FilterSpec()

    # Anahtar & değer normalizasyonu
    def _normalize_dict(d: Dict[str, Union[str, List[str]]]) -> Dict[str, Union[str, List[str]]]:
        out: Dict[str, Union[str, List[str]]] = {}
        for k, v in d.items():
            field = FIELD_MAP.get(k, k)
            if isinstance(v, list):
                out[field] = [_canon_value(field, x) for x in v]
            else:
                out[field] = _canon_value(field, v)
        return out

    def _normalize_ranges(rngs: Dict[str, RangeSpec]) -> Dict[str, RangeSpec]:
        out: Dict[str, RangeSpec] = {}
        for k, r in rngs.items():
            field = FIELD_MAP.get(k, k)
            out[field] = r
        return out

    spec.must = _normalize_dict(spec.must)
    spec.must_not = _normalize_dict(spec.must_not)
    spec.should = {FIELD_MAP.get(k, k): [_canon_value(FIELD_MAP.get(k, k), x) for x in v]
                   for k, v in spec.should.items()}
    spec.ranges = _normalize_ranges(spec.ranges)
    return spec

TEXT_EQ_FIELDS = {
    "marka_key","seri_key","model_key","konum_key","kasa_tipi_key",
    "cekis_key","renk_key","arac_durumu_key","vites_std","yakit_std"
}
NUM_FIELDS = {"fiyat_num","km_num","yil_num","tramer_num","Araç_Yası"}

def filterspec_to_qdrant(spec: FilterSpec) -> Optional[QFilter]:
    must: List[FieldCondition] = []
    should: List[FieldCondition] = []
    must_not: List[FieldCondition] = []

    # MUST koşulları (eşitlik ve range)
    for field, val in spec.must.items():
        if field in TEXT_EQ_FIELDS:
            if isinstance(val, list):
                for v in val:
                    must.append(FieldCondition(key=field, match=MatchValue(value=str(v))))
            else:
                must.append(FieldCondition(key=field, match=MatchValue(value=str(val))))
        elif field in NUM_FIELDS:
            try:
                fv = float(val) if not isinstance(val, list) else float(val[0])
                must.append(FieldCondition(key=field, range=Range(gte=fv, lte=fv)))
            except:
                pass

    # RANGE koşulları
    for field, r in spec.ranges.items():
        cond = {}
        if r.gte is not None: cond["gte"] = float(r.gte)
        if r.lte is not None: cond["lte"] = float(r.lte)
        if cond:
            must.append(FieldCondition(key=field, range=Range(**cond)))

    # MUST NOT koşulları
    for field, val in spec.must_not.items():
        if field in TEXT_EQ_FIELDS:
            if isinstance(val, list):
                for v in val:
                    must_not.append(FieldCondition(key=field, match=MatchValue(value=str(v))))
            else:
                must_not.append(FieldCondition(key=field, match=MatchValue(value=str(val))))

    # SHOULD koşulları (örnek: birden fazla şehir olabilir)
    for field, vals in spec.should.items():
        if field in TEXT_EQ_FIELDS:
            for v in vals:
                should.append(FieldCondition(key=field, match=MatchValue(value=str(v))))

    if not (must or should or must_not):
        return None
    return QFilter(must=must or None, should=should or None, must_not=must_not or None)
def make_payload_predicate(spec: FilterSpec):
    """
    Sparse sonuçları yerel olarak süzmek için predicate.
    """
    def ok(pl: Dict[str, Any]) -> bool:
        # must (eşitlik)
        for field, val in spec.must.items():
            pv = str(pl.get(field, "")).lower()
            if isinstance(val, list):
                if pv not in [str(x).lower() for x in val]:
                    return False
            else:
                if pv != str(val).lower():
                    return False

        # ranges
        for field, r in spec.ranges.items():
            v = pl.get(field)
            if v is None:
                return False
            try:
                vf = float(v)
            except:
                return False
            if r.gte is not None and vf < r.gte: return False
            if r.lte is not None and vf > r.lte: return False

        # must_not
        for field, val in spec.must_not.items():
            pv = str(pl.get(field, "")).lower()
            if isinstance(val, list):
                if pv in [str(x).lower() for x in val]:
                    return False
            else:
                if pv == str(val).lower():
                    return False

        # should: esnek (en az birini sağlasa iyi; sağlamasa da geçir)
        # Katılaştırmak istersen, en az bir should alanı sağlanmalı kontrolü ekleyebilirsin.
        return True
    return ok

def search_with_llm_filters(
    searcher: 'HybridSearcher',
    user_query: str,
    api_key: str,
    top_k: int = 30,
    model_name: str = "gpt-4o-mini",
    temperature: float = 0.0
):
    # 1) LLM'den FilterSpec üret
    spec = llm_parse_filters_all_fields(
        api_key=api_key,
        user_query=user_query,
        model_name=model_name,
        temperature=temperature
    )

    # 2) Qdrant Filter (dense için)
    qf = filterspec_to_qdrant(spec)

    # 3) Dense: Qdrant + filter; Sparse: yerel predicate
    if qf is None:
        dense = searcher.dense_search(user_query, f=None, top_k=top_k)
    else:
        res = searcher.client.search(
            collection_name=searcher.collection,
            query_vector=searcher.embedder.embed_query(user_query),
            query_filter=qf,
            limit=top_k,
            with_payload=True,
            with_vectors=False,
        )
        dense = [(r.id, float(r.score), r.payload or {}) for r in res]

    pred = make_payload_predicate(spec)
    sparse_all = searcher.sparse_search(user_query, f=None, top_k=top_k * 4)
    sparse = [t for t in sparse_all if pred(t[2])]

    # 4) RRF birleştir
    return searcher.rrf_merge(dense, sparse, top_k=top_k)


# ==========================================================
# 9) (İsteğe bağlı) LLM ile "madde madde öneri + link" çıktısı
# ==========================================================
from langchain_core.output_parsers import StrOutputParser

SELECTION_SYSTEM = """Aşağıda bir kullanıcının araç arama isteği ve aday listesi var.
Görev: Kullanıcının niyetine EN UYGUN en fazla 5 arabayı seç ve her birini madde işaretleriyle, kısa gerekçelerle ve tıklanabilir bağlantısıyla yaz.
Kurallar:
- Kullanıcı niyetindeki açık eşiklere (bütçe üst sınırı, yıl alt sınırı, km üst sınırı, tramer) uymayanları ele.
- Öncelik: bütçeyi aşmayanlar > yıl/km sınırına uyanlar > vites/yakıt/kasa/şehir uyumu > fiyat/performans.
- Format (sadece maddeler):
  • {{Baslik/Trim}} | {{Yıl}} | {{Km}} | {{Fiyat}} | {{Yakıt/Vites}} | {{Şehir}} | {{Neden kısa}} | [Link]
- Giriş/sonuç paragrafı yazma; sadece maddeler.
"""

SELECTION_HUMAN = """KULLANICI NİYETİ:
{user_query}

ADAY LİSTE (<=20) - JSON:
{candidates_json}
"""

selection_prompt = ChatPromptTemplate.from_messages([
    ("system", SELECTION_SYSTEM),
    ("human", SELECTION_HUMAN),
])

FIELD_KEEP_FOR_OUTPUT = [
    "id","marka","seri","model","yil_num","km_num","fiyat_num",
    "vites_tipi","yakit_tipi","kasa_tipi","konum","konum_key","url","baslik"
]

def simplify_candidates_for_output(results: List[tuple]) -> List[Dict[str, Any]]:
    simplified: List[Dict[str, Any]] = []
    for pid, score, pl in results[:20]:
        if not isinstance(pl, dict):
            continue
        row = {"id": pid, "score": float(score)}
        for k in FIELD_KEEP_FOR_OUTPUT:
            if k in pl:
                row[k] = pl[k]
        # eksik sayısalları None bırak (katı eleme yapma)
        for f in ["yil_num","km_num","fiyat_num"]:
            if f not in row or row[f] in ("", "—"):
                row[f] = None
        if "baslik" in row and isinstance(row["baslik"], str) and len(row["baslik"]) > 120:
            row["baslik"] = row["baslik"][:117] + "..."
        simplified.append(row)
    return simplified

def llm_selection_text(
    api_key: str,
    user_query: str,
    selection_results: List[tuple],
    model_name: str = "gpt-4o-mini",
    temperature: float = 0.0
) -> str:
    llm = ChatOpenAI(api_key=api_key, model=model_name, temperature=temperature)
    chain = selection_prompt | llm | StrOutputParser()
    cands = simplify_candidates_for_output(selection_results)
    import json as _json
    candidates_json = _json.dumps(cands, ensure_ascii=False)
    return chain.invoke({
        "user_query": user_query,
        "candidates_json": candidates_json
    })

In [16]:
searcher = HybridSearcher(client, collection, embedder)
results = search_with_llm_filters(
    searcher,
    user_query="ASTRA İSTİYORUM HEMEN ÇOK ACİL BANA ASTRA VER YÜZ BİN KM'DEN DÜŞÜK",
    api_key=API_KEY,
    top_k=50,
    model_name="gpt-4o-mini"
)
# filters = heuristic_parse(user_query)  
# results = searcher.search(user_query, None, top_k=20)

/tmp/ipykernel_22380/2225713883.py:426: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  res = self.client.search(


In [17]:
from typing import List, Dict, Any, Union
from pydantic import BaseModel, Field, conlist
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

In [18]:
from typing import List, Union, Dict, Any
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
import json

# ---------- 1) (Opsiyonel) Yapısal seçim modeli ----------
class PickedCar(BaseModel):
    id: Union[int, str] = Field(..., description="Orijinal point/record id")
    reason: str = Field(..., description="Seçme gerekçesi (kısa, somut)")

class Selection(BaseModel):
    selected: List[PickedCar] = Field(..., min_length=1, max_length=5)

parser = PydanticOutputParser(pydantic_object=Selection)

# ---------- 2) Sistem ve kullanıcı yönergeleri ----------
SYSTEM_STRUCTURED = """Aşağıda bir kullanıcının araç arama isteği ve bu isteğe göre bulunan aday arabalar var.
Görevin: Kullanıcının niyetine EN UYGUN en fazla 5 arabayı seç ve kısa gerekçeler yaz.
Kurallar:
- Kullanıcı niyetinden açık eşikler (bütçe üst sınırı, yıl alt sınırı, km üst sınırı) varsa bunları uygula.
- Öncelik: bütçeyi aşmayanlar > yıl/ km sınırına uyanlar > vites/yakıt/kasa/şehir uyumu > fiyat/performans.
- Uygun aday 5’ten azsa daha az dönebilirsin.
- ÇIKTIYI sadece {format_name} şemasına UYUMLU JSON olarak ver. Serbest metin yazma.
"""

HUMAN_STRUCTURED = """KULLANICI NİYETİ:
{user_query}

ADAY LİSTE (<=20):
{candidates_json}

ÇIKTI ŞEMASI:
{format_instructions}
"""

prompt_structured = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_STRUCTURED),
    ("human", HUMAN_STRUCTURED),
])

# ---------- 3) Serbest metin (kullanıcıya gösterilecek) ----------
SYSTEM_TEXT = """Aşağıda bir kullanıcının araç arama isteği ve aday listesi var.
Görev: Kullanıcının niyetine EN UYGUN en fazla 5 arabayı seç ve bunları liste gibi değil, doğal akışta bir satış asistanı gibi anlat.
Kurallar:
- Kullanıcı niyetindeki açık eşiklere (bütçe üst sınırı, yıl alt sınırı, km üst sınırı) uymayanları ele.
- Öncelik: bütçeyi aşmayanlar > yıl/km sınırına uyanlar > vites/yakıt/kasa/şehir uyumu > fiyat/performans.
- Eğer uygun aday 5’ten azsa daha az dönebilirsin.
- Üslup: Samimi, ikna edici ve yardımcı. Açıklamalar doğal dilde olacak, araç özelliklerini cümle içinde bütünleştir.
- Format:
  - Önce kısa bir giriş paragrafı (“Sizin için birkaç seçenek buldum” gibi).
  - Her aracı ayrı bir paragrafta tanıt. Örn: “2018 model Opel Astra 92 bin kilometrede, otomatik ve benzinli olmasıyla öne çıkıyor. Fiyatı 1 milyon 245 bin TL ve İstanbul’da. Bu aracın en büyük avantajı…” 
  - Aracın sonunda linki doğal cümlenin içine göm (“Detaylara buradan ulaşabilirsiniz: [Link]”).
  - Son olarak küçük bir kapanış paragrafı ekle (“İsterseniz bu seçeneklerden başlayabiliriz” gibi).
"""

HUMAN_TEXT = """KULLANICI NİYETİ:
{user_query}

ADAY LİSTE (<=20) - JSON:
{candidates_json}
"""

prompt_text = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_TEXT),
    ("human", HUMAN_TEXT),
])

# ---------- 4) LLM kurucu yardımcılar ----------
def build_llm(api_key: str, model_name: str = "gpt-4o-mini", temperature: float = 0.0) -> ChatOpenAI:
    return ChatOpenAI(api_key=api_key, model=model_name, temperature=temperature)

def build_selector_llm_structured(api_key: str, model_name: str = "gpt-4o-mini", temperature: float = 0.0):
    llm = build_llm(api_key, model_name, temperature)
    # Yapısal seçim (Pydantic) için zincir
    chain = prompt_structured | llm | parser
    return chain

def build_selector_llm_text(api_key: str, model_name: str = "gpt-4o-mini", temperature: float = 0.0):
    llm = build_llm(api_key, model_name, temperature)
    # Serbest metin (tam LLM cevabı) için zincir
    chain = prompt_text | llm | StrOutputParser()
    return chain

# ---------- 5) Aday sadeleştirme + filtreler ----------
FIELD_KEEP = [
    "id","marka","seri","model","yil_num","km_num","fiyat_num",
    "vites_tipi","yakit_tipi","kasa_tipi","sehir_key","konum","url","baslik"
]

REQUIRED_NUMERIC = ["yil_num", "km_num", "fiyat_num"]  # bu alanlar dolu/parse edilebilir olmalı

def is_valid_candidate(pl: Dict[str, Any]) -> bool:
    for f in REQUIRED_NUMERIC:
        if f not in pl or pl[f] in (None, "", "—"):
            return False
        # sayısal parse edilemiyorsa ele
        try:
            float(pl[f])
        except Exception:
            return False
    # URL/başlık da kontrol edilebilir
    if not pl.get("url"):
        return False
    return True

def simplify_candidates(results: list) -> List[Dict[str, Any]]:
    """
    results: HybridSearcher.search(...) çıktısı gibi (pid, score, payload) tuple listesi bekler.
    İlk 20 kaydı alır, zorunlu alanları kontrol eder, gösterim için kısaltır.
    """
    simplified: List[Dict[str, Any]] = []
    for pid, score, pl in results[:20]:
        if not isinstance(pl, dict):
            continue
        if not is_valid_candidate(pl):
            continue
        row = {"id": pid, "score": float(score)}
        for k in FIELD_KEEP:
            if k in pl:
                row[k] = pl[k]
        # başlık çok uzunsa kısalt
        if "baslik" in row and isinstance(row["baslik"], str) and len(row["baslik"]) > 120:
            row["baslik"] = row["baslik"][:117] + "..."
        simplified.append(row)
    return simplified

# ---------- 6) Dış API çağrıları ----------

# Not: Burada API_KEY'in global'de tanımlı olduğu varsayılmıştır.
# API_KEY = "..."  # kendi anahtarınızı sağlayın

def llm_select_top5_structured(user_query: str, results: list, api_key: str, model_name: str = "gpt-4o-mini") -> Selection:
    """(Opsiyonel) JSON-uyumlu yapısal seçim döner."""
    chain = build_selector_llm_structured(api_key=api_key, model_name=model_name)
    cands = simplify_candidates(results)
    candidates_json = json.dumps(cands, ensure_ascii=False)
    out: Selection = chain.invoke({
        "user_query": user_query,
        "candidates_json": candidates_json,
        "format_instructions": parser.get_format_instructions(),
        "format_name": "Selection"
    })
    return out

def llm_select_top5_text(user_query: str, results: list, api_key: str, model_name: str = "gpt-4o-mini") -> str:
    """
    Kullanıcıya gösterilecek TAM METİN (madde madde, neden + link) döner.
    Bu zincir, Pydantic parse ETMEZ; LLM'nin ürettiği tüm metin aynen gelir.
    """
    chain = build_selector_llm_text(api_key=api_key, model_name=model_name)
    cands = simplify_candidates(results)
    candidates_json = json.dumps(cands, ensure_ascii=False)
    text: str = chain.invoke({
        "user_query": user_query,
        "candidates_json": candidates_json
    })
    return text

In [19]:
full_text = llm_select_top5_text(
    user_query="ASTRA İSTİYORUM HEMEN ÇOK ACİL BANA ASTRA VER YÜZ BİN KM'DEN DÜŞÜK",
    results=results,
    api_key=API_KEY,
    model_name="gpt-4o-mini"
)
print(full_text)

Sizin için birkaç seçenek buldum. Hemen Astra arayışınıza uygun olanları inceleyelim.

İlk olarak, 2009 model Opel Astra 1.3 CDTI Essentia dikkat çekiyor. Bu araç, 345 bin kilometrede ve otomatik vites seçeneği ile geliyor. Fiyatı 399 bin TL olan bu Astra, hem konforlu bir sürüş sunuyor hem de dizel motoruyla yakıt tasarrufu sağlıyor. Diyarbakır'da bulunan bu aracı daha detaylı incelemek isterseniz, buradan ulaşabilirsiniz: [Detaylar](https://www.arabam.com/ilan/galeriden-satilik-opel-astra-1-3-cdti-essentia/hp-motor-s-dan-2009-astra-otomatik-bayan-araci/31915490).

Bir diğer seçenek ise 2013 model Opel Astra 1.3 CDTI Edition. Bu araç 313 bin kilometrede ve düz vites seçeneği ile geliyor. Fiyatı 590 bin TL olan bu Astra, hem şık tasarımı hem de performansıyla öne çıkıyor. Konya'da bulunan bu araca göz atmak isterseniz, detaylara buradan ulaşabilirsiniz: [Detaylar](https://www.arabam.com/ilan/galeriden-satilik-opel-astra-1-3-cdti-edition/hatasiz-boyasiz-opel-astra/32862417).

Maalesef, 